<a href="https://colab.research.google.com/github/calicartels/XAI--Explainable-Techniques-II/blob/main/Explainable_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explainable Techniques II


In [ ]:

!pip install numpy==1.25.2 pandas==2.0.3 scikit-learn==1.2.2 shap==0.45.1
!pip install git+https://github.com/MaximeJumelle/ALEPython.git@dev#egg=alepython

In [ ]:

# Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Models
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

# XAI
import shap
from alepython import ale_plot
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import permutation_importance

np.random.seed(1)

In [ ]:
# for correlation matrix

X, y = shap.datasets.linnerud()
X = pd.DataFrame(X, columns=['Chins', 'Situps', 'Jumps'])
y = pd.DataFrame(y, columns=['Weight', 'Waist', 'Pulse'])

In [ ]:
plt.figure(figsize=(8, 5))
correlation_matrix = X.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Features')
plt.show()

The correlation matrix reveals several important relationships:
- Strong positive correlation (0.70) between Chins and Situps
- Strong positive correlation (0.67) between Situps and Jumps
- Moderate positive correlation (0.50) between Jumps and Chins

In [ ]:


# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y['Weight'], test_size=0.2, random_state=42)



In [ ]:
# Training GradientBoosting
model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)



In [ ]:

# Lets Check PDP of features Chins and Situps
# Partial Dependence Plots (PDP)
features_to_plot = [0, 1]
PartialDependenceDisplay.from_estimator(
    model,
    X_train,
    features=features_to_plot,
    grid_resolution=50
)
plt.show()



Chins:
-	Shows a sharp drop initially for lower values of Chins, then stabilizes with slight fluctuations.
-	After the initial drop, increasing the number of chin-ups has a limited effect on the target variable.
-	Suggests that low values of Chins might initially reduce the target variable, but higher values have less impact.

Situps:
- Displays a strong initial positive partial dependence, followed by a steep decline as the number of sit-ups increases.
- After the initial drop, the curve fluctuates but stays generally lower than at the start.
- Indicates that higher numbers of sit-ups have a negative effect on the target variable, likely due to associations with physical endurance.


In [ ]:
# SHAP values
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

# SHAP summary
shap.summary_plot(shap_values, X_test)
plt.show()

Chins:
- Moderate impact on model output, with both positive and negative SHAP values.
- Higher Chins values tend to push predictions slightly negatively, while lower values may have a neutral or slightly positive effect.
- Shows that Chins has a mixed influence on the target, depending on its value.

Situps:
- Strong negative SHAP values for higher values of Situps, indicating a consistent negative impact on model predictions.
- Lower values of Situps have minimal influence, while higher values drive the target variable down significantly.
- Suggests that high numbers of sit-ups reduce the target variable’s predicted outcome.

Jumps:
- Low overall impact on predictions, with both positive and negative SHAP values, indicating minimal influence on the target.
- Some negative SHAP values for higher Jumps values, but the effect is minor.
- Indicates that Jumps contributes little to the model’s predictions in either direction.